# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Import weather data from part I
csv_path = os.path.join("..", "output_data", "clean_city_data.csv")

clean_city_data_df = pd.read_csv(csv_path)
clean_city_data_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Date
0,klaksvik,74.18426,-7.01743,51.80,93,86,10.29,fo,1600398913
1,calbuco,-43.10707,-72.17928,44.60,93,90,2.24,cl,1600398678
2,itacoatiara,-3.00747,-58.14442,76.82,79,2,1.86,br,1600398736
3,bluff,-89.22161,168.41832,51.01,76,100,25.01,nz,1600398914
4,ushuaia,-70.17430,-43.10707,30.20,86,90,17.22,ar,1600398914
...,...,...,...,...,...,...,...,...,...
549,nova esperanca,-23.05727,-52.12948,69.94,92,93,1.63,br,1600399084
550,salekhard,65.16185,66.16434,44.26,86,100,12.12,ru,1600399084
551,belaya gora,67.16683,145.36105,46.26,88,100,15.03,ru,1600399084
552,aktau,50.12450,74.18426,48.20,61,18,11.18,kz,1600399085


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# SET UP LAT LONG PAIRS
Coords1 = clean_city_data_df[['Latitude', 'Longitude']]

# Set up a list of humidities
Humidity = clean_city_data_df['Humidity']
Humidity

# CONFIGURE GMAPS
gmaps.configure(api_key=g_key)

# Customize Layout
figure_layout = {
    'width': '800px',
    'height': '600px'
}

# store the figure in a variable so we can continue to add layers to the same map
Myfig = gmaps.figure(layout=figure_layout, center=(0, 0), zoom_level=2)

# Create a marker layer using our coordinates
heat_layer = gmaps.heatmap_layer(Coords1, weights=Humidity)

# Add the layer to the map
Myfig.add_layer(heat_layer)

# Save and display the figure with the newly added layer
Myfig


Figure(layout=FigureLayout(height='600px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#  Selecting ideal cities. Temp <85, >65, Wind speed <15, Cloudiness <10, Humidity<55
ideal_weather_cities_df = clean_city_data_df.loc[(clean_city_data_df["Max_Temp"]<85) & (clean_city_data_df["Max_Temp"]>65) & 
                                                        (clean_city_data_df["Wind_Speed"]<15) & (clean_city_data_df["Cloudiness"]<10) & 
                                                        (clean_city_data_df["Humidity"]<55) & (clean_city_data_df["Humidity"]>35) ]

print(f"No. of ideal cities: {ideal_weather_cities_df['City'].count()}")
ideal_weather_cities_df

No. of ideal cities: 10


,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Date
63,laguna,-29.07221,-48.11952,77.00,47,1,12.75,br,1600398930
69,airai,17.04233,148.36852,80.53,40,3,3.89,pw,1600398931
107,yulara,-24.05976,126.31374,66.20,42,0,11.41,au,1600398942
261,pompeu,-19.04731,-45.11205,70.48,38,0,5.17,br,1600398988
309,guerrero negro,18.04482,-128.31872,82.89,42,0,8.72,mx,1600399006
313,nalut,31.07719,10.02490,69.42,49,0,10.33,ly,1600399007
339,moussoro,14.03486,16.03984,84.18,50,0,7.25,td,1600399015
424,marzuq,24.05976,19.04731,76.37,48,0,1.19,ly,1600399044
475,arinos,-16.03984,-46.11454,68.83,42,0,1.70,br,1600399062
494,tanout,15.03735,9.02241,83.53,50,0,12.10,ne,1600399068


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create new hotel df, and create extra columns
hotel_df = ideal_weather_cities_df.copy()
hotel_df['Country'] = hotel_df['Country_Code']
hotel_df["Hotel Name"] = ""
hotel_df["Hotel_Lat"] = ""
hotel_df["Hotel_Long"] = ""

hotel_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Date,Country,Hotel Name,Hotel_Lat,Hotel_Long
63,laguna,-29.07221,-48.11952,77.00,47,1,12.75,br,1600398930,br,,,
69,airai,17.04233,148.36852,80.53,40,3,3.89,pw,1600398931,pw,,,
107,yulara,-24.05976,126.31374,66.20,42,0,11.41,au,1600398942,au,,,
261,pompeu,-19.04731,-45.11205,70.48,38,0,5.17,br,1600398988,br,,,
309,guerrero negro,18.04482,-128.31872,82.89,42,0,8.72,mx,1600399006,mx,,,
313,nalut,31.07719,10.02490,69.42,49,0,10.33,ly,1600399007,ly,,,
339,moussoro,14.03486,16.03984,84.18,50,0,7.25,td,1600399015,td,,,
424,marzuq,24.05976,19.04731,76.37,48,0,1.19,ly,1600399044,ly,,,
475,arinos,-16.03984,-46.11454,68.83,42,0,1.70,br,1600399062,br,,,
494,tanout,15.03735,9.02241,83.53,50,0,12.10,ne,1600399068,ne,,,


In [6]:
# geocoordinates for the base of our search
target_location = "29.714033,-95.406480" # coordinates for Rice University

# distance, IN METERS, within which the place results must live
target_radius = 20000

# keyword term to be matched against all of Google's indexed content
target_keyword = "Hotel"

# type of establishment that we'd like to use to filter our place results
target_type = "Hotel"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a dictionary to store all query parameters
parameters = {
#     "location": target_location,
    "keyword": target_keyword,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# # # run a request using our params dictionary
# run a request using our params dictionary
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    print(f"Searching for hotel in city with lat lng {lat}, {lng}...")
# update params dictionary
    parameters['location'] = f"{lat},{lng}"
    hotel_info = requests.get(base_url, params=parameters)
    hotel_info_json = hotel_info.json()
# Incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, 'Hotel Name'] =  hotel_info_json['results'][0]['name']
        hotel_df.loc[index, 'Hotel_Lat'] =  hotel_info_json['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel_Long'] =  hotel_info_json['results'][0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
print(f"No. of Hotels found in our ideal cities: {hotel_df['Hotel Name'].count()}")
hotel_df

Searching for hotel in city with lat lng -29.07221, -48.11952000000001...
Missing field/result... skipping.
Searching for hotel in city with lat lng 17.042330000000003, 148.36852000000005...
Missing field/result... skipping.
Searching for hotel in city with lat lng -24.059760000000004, 126.31374...
Missing field/result... skipping.
Searching for hotel in city with lat lng -19.047310000000003, -45.11205...
Searching for hotel in city with lat lng 18.04482, -128.31871999999998...
Missing field/result... skipping.
Searching for hotel in city with lat lng 31.07719, 10.0249...
Searching for hotel in city with lat lng 14.034860000000002, 16.039839999999998...
Missing field/result... skipping.
Searching for hotel in city with lat lng 24.059760000000004, 19.047310000000003...
Missing field/result... skipping.
Searching for hotel in city with lat lng -16.039839999999998, -46.11454000000001...
Searching for hotel in city with lat lng 15.037350000000002, 9.02241...
Missing field/result... skippin

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Date,Country,Hotel Name,Hotel_Lat,Hotel_Long
63,laguna,-29.07221,-48.11952,77.00,47,1,12.75,br,1600398930,br,,,
69,airai,17.04233,148.36852,80.53,40,3,3.89,pw,1600398931,pw,,,
107,yulara,-24.05976,126.31374,66.20,42,0,11.41,au,1600398942,au,,,
261,pompeu,-19.04731,-45.11205,70.48,38,0,5.17,br,1600398988,br,Nita Palace Hotel,-19.2189,-44.9991
309,guerrero negro,18.04482,-128.31872,82.89,42,0,8.72,mx,1600399006,mx,,,
313,nalut,31.07719,10.02490,69.42,49,0,10.33,ly,1600399007,ly,Petro serv catering camp 1,31.0294,10.116
339,moussoro,14.03486,16.03984,84.18,50,0,7.25,td,1600399015,td,,,
424,marzuq,24.05976,19.04731,76.37,48,0,1.19,ly,1600399044,ly,,,
475,arinos,-16.03984,-46.11454,68.83,42,0,1.70,br,1600399062,br,Big Hotel,-15.9106,-46.1131
494,tanout,15.03735,9.02241,83.53,50,0,12.10,ne,1600399068,ne,,,


In [7]:
# Clean dataframe
# hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df['Hotel Name'].dropna(inplace=True)

hotel_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Date,Country,Hotel Name,Hotel_Lat,Hotel_Long
63,laguna,-29.07221,-48.11952,77.00,47,1,12.75,br,1600398930,br,,,
69,airai,17.04233,148.36852,80.53,40,3,3.89,pw,1600398931,pw,,,
107,yulara,-24.05976,126.31374,66.20,42,0,11.41,au,1600398942,au,,,
261,pompeu,-19.04731,-45.11205,70.48,38,0,5.17,br,1600398988,br,Nita Palace Hotel,-19.2189,-44.9991
309,guerrero negro,18.04482,-128.31872,82.89,42,0,8.72,mx,1600399006,mx,,,
313,nalut,31.07719,10.02490,69.42,49,0,10.33,ly,1600399007,ly,Petro serv catering camp 1,31.0294,10.116
339,moussoro,14.03486,16.03984,84.18,50,0,7.25,td,1600399015,td,,,
424,marzuq,24.05976,19.04731,76.37,48,0,1.19,ly,1600399044,ly,,,
475,arinos,-16.03984,-46.11454,68.83,42,0,1.70,br,1600399062,br,Big Hotel,-15.9106,-46.1131
494,tanout,15.03735,9.02241,83.53,50,0,12.10,ne,1600399068,ne,,,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations)
Myfig.add_layer(hotel_layer)

# Save and display figure
Myfig

Figure(layout=FigureLayout(height='600px', width='800px'))